# `Whole-Genome-Sequence-Analysis`

In [24]:
#Tool Installation
#!conda install -c bioconda fastqc
#!conda install -c bioconda multiqc
#!conda install -c bioconda trimmmomatic
#!conda install -c bioconda bwa
#!conda install -c bioconda samtools
#!conda install -c bioconda gatk4

## `1. Quality Analysis`

In [3]:
!mkdir -p fastqc_raw multiqc_raw

In [7]:
!fastqc -o fastqc_raw/ data/*.gz

In [8]:
!multiqc fastqc_raw/ -o multiqc_raw/

## `2. Quality Control`

In [6]:
!mkdir -p trim_data 

In [9]:
!trimmomatic PE -phred33 data/SRR062634_1.fastq.gz data/SRR062634_2.fastq.gz trim_data/SRR062634_1_paired.fq.gz trim_data/SRR062634_1_unpaired.fq.gz trim_data/SRR062634_2_paired.fq.gz trim_data/SRR062634_2_unpaired.fq.gz ILLUMINACLIP:"/home/nishant.shekhar/miniconda3/pkgs/trimmomatic-0.39-hdfd78af_2/share/trimmomatic-0.39-2/adapters/TruSeq3-PE.fa":2:30:10 LEADING:3 TRAILING:3 SLIDINGWINDOW:4:15 MINLEN:36

In [ ]:
# Run fastqc and mutiqc to check the quality of the reads after trimming 

In [10]:
!fastqc trim_data/*.gz -o trim_data/

In [11]:
!multiqc trim_data/ -o trim_data

## `3. Indexing with BWA`

In [ ]:
#Download the genome file 

In [12]:
!wget https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/001/405/GCF_000001405.40_GRCh38.p14/GCF_000001405.40_GRCh38.p14_genomic.fna.gz

In [ ]:
#Download the annotation file 

In [13]:
!wget https://ftp.ncbi.nlm.nih.gov/genomes/all/GCF/000/001/405/GCF_000001405.40_GRCh38.p14/GCF_000001405.40_GRCh38.p14_genomic.gff.gz

In [15]:
# Unzipping both genome and h=gtf file

In [1]:
!gzip -d GCF_000001405.40_GRCh38.p14_genomic.fna.gz
!gzip -d GCF_000001405.40_GRCh38.p14_genomic.gff.gz

In [3]:
!mv GCF_000001405.40_GRCh38.p14_genomic.fna hg38.fa

mv: cannot stat 'GCF_000001405.40_GRCh38.p14_genomic.fna': No such file or directory


In [4]:
!mkdir -p genome

In [6]:
!mv hg38.fa genome/ # Moving the genome file to directory genome

In [6]:
!bwa index genome/hg38.fa

## `4. Mapping/Alignment with BWA`

In [5]:
!bwa mem -t 4 -R "@RG\tID:SRR062634\tPL:ILLUMINA\tSM:SRR062634" \
    genome/hg38.fa trim_data/SRR062634_1_paired.fq.gz trim_data/SRR062634_2_paired.fq.gz \
    | samtools view -Sb - > SRR062634.bam

## `5.Sorting the bam file with SAMTOOLS`

In [17]:
!samtools sort SRR062634.bam -o SRR062634_sort.bam

## `6.Removing the MarkDuplicates With tool GATK4`

In [20]:
#We can use Multiple Options \
    #gatk MarkDuplicate
    #gatk MarkDuplicatesSpark

In [14]:
!gatk MarkDuplicatesSpark -I SRR062634_sort.bam -O marked_duplicates.bam -M marked_dup_metrics.txt --conf 'spark.executor.cores=5'

In [19]:
## Flags explanation for MarkDupliactesSpark : https://gatk.broadinstitute.org/hc/en-us/articles/4414594330011-MarkDuplicatesSpark

## `7.BQSR(Base Quality Score Recalibration)`

In [ ]:
!gatk BaseRecalibrator -R genome/hg38.fa -I marked_duplicates.bam --known-sites dbsnp_138.hg38.vcf --known-sites  Mills_and_1000G_gold_standard.indels.hg38.sites.vcf --known-sites CosmicVariants.vcf -O recal_data.table

## If you get error then the do the following mentioned thing 
The dbsnp_138, Mill_and_1000G and Cosariants.vcf vcfs files must be sorted according to the reference fasta dictionary using `gatk’s SortVcf`

In [ ]:
!gatk ApplyBQSR -bqsr recal_data.table -R genome/hg38.fa -I marked_duplicates.bam -O sample_recal_reads.bam --create-output-bam-index true

## `8.Variant Calling using Haplotype Caller(Germline Variants)`

In [ ]:
!gatk HaplotypeCaller -R genome/hg38.fa -I sample_recal_reads.bam -O raw_variants.vcf